In [1]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.2.1+cu121
torchvision version: 0.17.1+cpu


In [2]:
import matplotlib.pyplot as plt
import os
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary
    
from modular.going_modular import data_setup, engine
from helper_functions import download_data, set_seeds, plot_loss_curves    

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
def create_effnetb2_model(num_classes, seed: int = 42):
    """Creates an EfficientNetB2 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head.
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNetB2 feature extractor model.
        transforms (torchvision.transforms): EffNetB2 image transforms.
    """
    # 1, 2, 3. Create EffNetB2 pretrained weights, transforms and model
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    # 4. Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # 5. Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes),
    )

    return model, transforms

In [5]:
effnetb2_food101, effnetb2_transforms = create_effnetb2_model(num_classes=101)

In [6]:
effnetb2_transforms

ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [7]:
# Check model classifier head
effnetb2_food101.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=101, bias=True)
)

In [8]:
from torchinfo import summary

# Get a summary of EffNetB2 feature extractor for Food101 with 101 output classes (uncomment for full output)
summary(
    effnetb2_food101,
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 101]             --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

In [9]:
# Create Food101 training data transforms (only perform data augmentation on the training images)
food101_train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.TrivialAugmentWide(),
        effnetb2_transforms,
    ]
)

In [10]:
print(f"Training transforms:\n{food101_train_transforms}\n")
print(f"Testing transforms:\n{effnetb2_transforms}")

Training transforms:
Compose(
    TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
)

Testing transforms:
ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


In [12]:
from torchvision import datasets

# Setup data directory
from pathlib import Path

data_dir = Path("data")

# Get training data (~750 images x 101 food classes)
train_data = datasets.Food101(
    root=data_dir,  # path to download data to
    split="train",  # dataset split to get
    transform=food101_train_transforms,  # perform data augmentation on training data
    download=False,
)  # want to download?

# Get testing data (~250 images x 101 food classes)
test_data = datasets.Food101(
    root=data_dir,
    split="test",
    transform=effnetb2_transforms,  # perform normal EffNetB2 transforms on test data
    download=False,
)

In [13]:
# Get Food101 class names
food101_class_names = train_data.classes

# View the first 20
food101_class_names[:20]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla']

In [14]:
def split_dataset(
    dataset: torchvision.datasets, split_size: float = 0.5, seed: int = 42
):
    """Randomly splits a given dataset into two proportions based on split_size and seed.

    Args:
        dataset (torchvision.datasets): A PyTorch Dataset, typically one from torchvision.datasets.
        split_size (float, optional): How much of the dataset should be split?
            E.g. split_size=0.2 means there will be a 20% split and an 80% split. Defaults to 0.2.
        seed (int, optional): Seed for random generator. Defaults to 42.

    Returns:
        tuple: (random_split_1, random_split_2) where random_split_1 is of size split_size*len(dataset) and
            random_split_2 is of size (1-split_size)*len(dataset).
    """
    # Create split lengths based on original dataset length
    length_1 = int(len(dataset) * split_size)  # desired length
    length_2 = len(dataset) - length_1  # remaining length

    # Print out info
    print(
        f"[INFO] Splitting dataset of length {len(dataset)} into splits of size: {length_1} ({int(split_size*100)}%), {length_2} ({int((1-split_size)*100)}%)"
    )

    # Create splits with given random seed
    random_split_1, random_split_2 = torch.utils.data.random_split(
        dataset, lengths=[length_1, length_2], generator=torch.manual_seed(seed)
    )  # set the random seed for reproducible splits
    return random_split_1, random_split_2

In [15]:
# Create training 50% split of Food101
train_data_food101_50_percent, _ = split_dataset(dataset=train_data, split_size=0.2)

# Create testing 20% split of Food101
test_data_food101_50_percent, _ = split_dataset(dataset=test_data, split_size=0.2)

len(train_data_food101_50_percent), len(test_data_food101_50_percent)

[INFO] Splitting dataset of length 75750 into splits of size: 15150 (20%), 60600 (80%)
[INFO] Splitting dataset of length 25250 into splits of size: 5050 (20%), 20200 (80%)


(15150, 5050)

In [16]:
os.cpu_count()

16

In [17]:
BATCH_SIZE = 32
NUM_WORKERS = 2 if os.cpu_count() <= 4 else 4 # this value is very experimental and will depend on the hardware you have available, Google Colab generally provides 2x CPUs

# Create Food101 50 percent training DataLoader
train_dataloader_food101_50_percent = torch.utils.data.DataLoader(train_data_food101_50_percent,
                                                                  batch_size=BATCH_SIZE,
                                                                  shuffle=True,
                                                                  num_workers=NUM_WORKERS)
# Create Food101 50 percent testing DataLoader
test_dataloader_food101_50_percent = torch.utils.data.DataLoader(test_data_food101_50_percent,
                                                                 batch_size=BATCH_SIZE,
                                                                 shuffle=False,
                                                                 num_workers=NUM_WORKERS)

In [18]:
from modular.going_modular import engine

# Setup optimizer
optimizer = torch.optim.Adam(params=effnetb2_food101.parameters(),
                             lr=1e-3)

# Setup loss function
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1) # throw in a little label smoothing because so many classes

# Want to beat original Food101 paper with 20% of data, need 56.4%+ acc on test dataset
set_seeds()    
effnetb2_food101_results = engine.train(model=effnetb2_food101,
                                        train_dataloader=train_dataloader_food101_50_percent,
                                        test_dataloader=test_dataloader_food101_50_percent,
                                        optimizer=optimizer,
                                        loss_fn=loss_fn,
                                        epochs=10,
                                        device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.6332 | train_acc: 0.2871 | test_loss: 2.7667 | test_acc: 0.4929
Epoch: 2 | train_loss: 2.8626 | train_acc: 0.4396 | test_loss: 2.4654 | test_acc: 0.5369
Epoch: 3 | train_loss: 2.6583 | train_acc: 0.4842 | test_loss: 2.3555 | test_acc: 0.5653
Epoch: 4 | train_loss: 2.5506 | train_acc: 0.5094 | test_loss: 2.3029 | test_acc: 0.5751
Epoch: 5 | train_loss: 2.4958 | train_acc: 0.5271 | test_loss: 2.2808 | test_acc: 0.5782
Epoch: 6 | train_loss: 2.4535 | train_acc: 0.5327 | test_loss: 2.2665 | test_acc: 0.5853
Epoch: 7 | train_loss: 2.4059 | train_acc: 0.5464 | test_loss: 2.2442 | test_acc: 0.5858
Epoch: 8 | train_loss: 2.3875 | train_acc: 0.5503 | test_loss: 2.2414 | test_acc: 0.5921
Epoch: 9 | train_loss: 2.3636 | train_acc: 0.5552 | test_loss: 2.2303 | test_acc: 0.5977
Epoch: 10 | train_loss: 2.3440 | train_acc: 0.5616 | test_loss: 2.2453 | test_acc: 0.5849


In [1]:
from helper_functions import plot_loss_curves

# Check out the loss curves for FoodVision Big
plot_loss_curves(effnetb2_food101_results)

NameError: name 'effnetb2_food101_results' is not defined

In [ ]:
from going_modular.going_modular import utils

# Create a model path
effnetb2_food101_model_path = "pretrained_effnetb2_feature_extractor_food101_50_percent.pth" 

# Save FoodVision Big model
utils.save_model(model=effnetb2_food101,
                 target_dir="models",
                 model_name=effnetb2_food101_model_path)